<a href="https://colab.research.google.com/github/MarylouBer/MLD_Data_Engineering_portfolio/blob/main/Monthly_Reporting_Automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:

### PART 1 ---> creating the report

from google.colab import files, drive
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# Step 1: Upload Excel file
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# Step 2: Read data, skipping 3 rows
df_data = pd.read_excel(file_name, skiprows=3)

# Step 3: Rename month-related columns manually
rename_map = {
    'Unnamed: 44': 'Actual',
    'Unnamed: 45': 'Budget',
    'Unnamed: 46': 'LE',
    'Unnamed: 47': 'Planning',
    'Unnamed: 48': 'Budget vs LE',
}
df_data.rename(columns=rename_map, inplace=True)

# Step 4: Find Employees column
employee_col = [col for col in df_data.columns if 'Employee' in str(col)]
if not employee_col:
    raise ValueError("❌ Could not find an 'Employees' column.")
employee_col = employee_col[0]
print(f"✅ Found Employees column: {employee_col}")

# Step 5: Filter final DataFrame
month_columns = list(rename_map.values())
filtered_df = df_data[[employee_col] + month_columns]

# Step 6: Add empty "Comment" column as the last column
filtered_df['Comment'] = ''

# Step 7: Mount Google Drive
drive.mount('/content/drive')

# Step 8: Save to Google Drive (first pass, without formatting)
#output_path = '/content/drive/MyDrive/June_2025_report.xlsx'
output_path = '/content/drive/MyDrive/Finance_Systems_Office/Controlling/Monthly_Controlling_Review/Reports/Accruals_Reports/2025/AccrualsConfirmationDirectors/June_2025_accruals.xlsx'
filtered_df.to_excel(output_path, index=False)

# Step 9: Load existing workbook and select active sheet for transformation tasks
wb = load_workbook(output_path)
ws = wb.active

# Step 8: Setting up the color palette to highlight specific items

yellow_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
grey_fill = PatternFill(start_color="C0C0C0", end_color="C0C0C0", fill_type="solid")
red_fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")
pale_yellow_fill = PatternFill(start_color="FFFFE0", end_color="FFFFE0", fill_type="solid")

# Step 9: Highlight specific Values in yellow

highlight_ids = ['1001010034', '1001010035', '1001010036', '1001010037', '1001010038', '1001010039']

for row in ws.iter_rows(min_row=2, max_row=ws.max_row, max_col=ws.max_column):
    emp_id = str(row[0].value).strip()
    if emp_id in highlight_ids:
        for cell in row:
            cell.fill = yellow_fill

#Step 10: Grey out and clear all columns except 1st for rows with "xxx" or "yyy"
for row in ws.iter_rows(min_row=2, max_row=ws.max_row, max_col=ws.max_column):
    row_values = [str(cell.value).lower() if cell.value else '' for cell in row]
    if any(keyword in val for val in row_values for keyword in ['xxx', 'yyy']):
        for i, cell in enumerate(row):
            if i != 0:  # Keep the first column (Employee ID)
                cell.value = None
            cell.fill = pale_yellow_fill

# Step 14: Color the header of the "Comment" column grey
for cell in ws[1]:
    if cell.value == 'Comment':
        cell.fill = grey_fill
        break

# Step 15: Color the header of the "Planning" column red
for cell in ws[1]:
    if cell.value == 'Planning':
        cell.fill = red_fill
        break

# Step 11: Collect rows to delete (row numbers)
rows_to_delete = []

for row in ws.iter_rows(min_row=2, max_row=ws.max_row, max_col=ws.max_column):
    first_cell_value = str(row[0].value).lower() if row[0].value else ''
    if first_cell_value.startswith(('hello', 'hello1')):
        rows_to_delete.append(row[0].row)

# Step 12: Delete rows
for row_idx in sorted(rows_to_delete, reverse=True):
    ws.delete_rows(row_idx)

# Step 13: Auto-adjust column widths, make 'Comment' column three times as wide
for column_cells in ws.columns:
    max_length = 0
    column_letter = column_cells[0].column_letter

    for cell in column_cells:
        try:
            cell_value = str(cell.value)
            if len(cell_value) > max_length:
                max_length = len(cell_value)
        except:
            pass

    if column_cells[0].value == 'Comment':
        adjusted_width = (max_length + 2) * 3
    else:
        adjusted_width = max_length + 2

    ws.column_dimensions[column_letter].width = adjusted_width

# Save workbook again
wb.save(output_path)

print(f"File saved with highlighted rows, adjusted column widths, and 'Comment' column styled at:\n{output_path}")



### PART 2 ---> Sending email

import smtplib
from email.mime.text import MIMEText
import getpass

# Step 1: Prompt for password securely
email_address = "xxx.xxx@test.com"
password = getpass.getpass("Enter your Gmail password (or App Password): ")

# Step 2: Compose email
msg = MIMEText("""Hi all,

Please review the monthly report:

https://drive.google.com/drive/my-drive

Thanks
""")

msg["Subject"] = "Monthly Report"
msg["From"] = email_address
msg["To"] = "xxx.xxx@xxx.com"

# Step 3: Send email

with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
    server.login(email_address, password)
    server.send_message(msg)

print("Email sent!")




Saving Externals Costs Overview - Actuals & Plan (2).xlsx to Externals Costs Overview - Actuals & Plan (2) (2).xlsx
✅ Found Employees column: Employees


<ipython-input-4-2756271469>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Comment'] = ''


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
File saved with highlighted rows, adjusted column widths, and 'Comment' column styled at:
/content/drive/MyDrive/Finance_Systems_Office/Controlling/Monthly_Controlling_Review/Reports/Accruals_Reports/2025/AccrualsConfirmationDirectors/June_2025_accruals.xlsx


KeyboardInterrupt: Interrupted by user